In [2]:
import re
import pickle
import random as rd
import pandas as pd
import torch.nn as nn
import seaborn as sns
from tqdm import tqdm
from nltk.corpus import stopwords
from tokenizers import Tokenizer,models,trainers
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import torch
from torch.utils.data import DataLoader,Dataset

In [3]:
lenta_news = pd.read_csv('lenta_news.csv')
lenta_news

,title,text,topic
0,МВД создало бюро по борьбе с отмыванием денег,В МВД России создано оперативно-розыскное бюро...,Экономика
1,Правительство не будет занимать деньги у ЦБ,Правительство не будет прибегать к заимствован...,Экономика
2,"ВВС США заказали 60 ""Боингов"" для вторжения в ...",Компания Boeing получит пятилетний контракт на...,Экономика
3,Правительственная комиссия не дала ExxonMobil ...,Результаты конкурса на право освоения месторож...,Экономика
4,Неимущим для получения пособия придется расста...,В доходах малоимущих россиян будут учитываться...,Экономика
...,...,...,...
23944,"Борис Гребенщиков награжден орденом ""За заслуг...","Лидер легендарной группы ""Аквариум"" Борис Греб...",Культура
23945,Мадонна и Траволта названы худшими актерами года,"24 марта, накануне вручения премии Американско...",Культура
23946,Габриэль Гарсиа Маркес простил Голливуд ради д...,Габриэль Гарсиа Маркес,Культура
23947,Аль Пачино отстоял в суде свое право на детей,Знаменитый американский актер Аль Пачино добил...,Культура


In [31]:
rbk_news = pd.read_csv('rbk_news.csv',header = None)
rbk_news

,0,1
0,США обеспокоены участием военных КНДР в военно...,Политика
1,"Пхеньян, по ее словам «не помогает» в урегулир...",Политика
2,США «по-прежнему обеспокоены прямым участием» ...,Политика
3,«Переброска на территорию России военных [КНДР...,Политика
4,\n\n\n\n\n\n\n Минобороны п...,Политика
...,...,...
1170,\n\n\n\n\n\n\n ЦБ подумает ...,Финансы
1171,В отличие от стратегий автоследования в ОПИФе ...,Финансы
1172,\n\n\n\n\n\n\n Индекс Мосби...,Финансы
1173,Идею привлечения авторов инвестиционных блогов...,Финансы


In [17]:
rbk_news.iloc[:,1].unique()

array(['Политика', 'Экономика', 'Бизнес', 'Технологии и медиа', 'Финансы'],
      dtype=object)

In [16]:
lenta_news.iloc[:,2].unique()

array(['Экономика', 'Интернет и СМИ', 'Спорт', 'Культура'], dtype=object)

In [37]:
rbk_news.loc[rbk_news[1] == 'Технологии и медиа',1] = 'Интернет и СМИ'

In [42]:
rbk_news[1].unique()

array(['Политика', 'Экономика', 'Бизнес', 'Интернет и СМИ', 'Финансы'],
      dtype=object)

In [46]:
lenta_news.iloc[:,2].unique()

array(['Экономика', 'Интернет и СМИ', 'Спорт', 'Культура'], dtype=object)

In [49]:
all_classes = list(set(rbk_news[1].unique().tolist() + lenta_news.iloc[:,2].unique().tolist()))

In [51]:
print(len(all_classes))
all_classes

7


['Экономика',
 'Бизнес',
 'Финансы',
 'Культура',
 'Интернет и СМИ',
 'Спорт',
 'Политика']

In [65]:
lent_text_col = pd.DataFrame({'Text':[lenta_news.iloc[row,0]+ ' ' +lenta_news.iloc[row,1] for row in range(len(lenta_news))]})

In [ ]:
lenta_news.insert(loc = 0,column = 'Text',value = lent_text_col['Text'])

In [80]:
lenta_news.drop(['title','text'],axis = 1,inplace = True)

In [81]:
lenta_news

,Text,topic
0,МВД создало бюро по борьбе с отмыванием денег ...,Экономика
1,Правительство не будет занимать деньги у ЦБ Пр...,Экономика
2,"ВВС США заказали 60 ""Боингов"" для вторжения в ...",Экономика
3,Правительственная комиссия не дала ExxonMobil ...,Экономика
4,Неимущим для получения пособия придется расста...,Экономика
...,...,...
23944,"Борис Гребенщиков награжден орденом ""За заслуг...",Культура
23945,Мадонна и Траволта названы худшими актерами го...,Культура
23946,Габриэль Гарсиа Маркес простил Голливуд ради д...,Культура
23947,Аль Пачино отстоял в суде свое право на детей ...,Культура


In [83]:
rbk_news.columns = ['Text','topic']

In [84]:
rbk_news

,Text,topic
0,США обеспокоены участием военных КНДР в военно...,Политика
1,"Пхеньян, по ее словам «не помогает» в урегулир...",Политика
2,США «по-прежнему обеспокоены прямым участием» ...,Политика
3,«Переброска на территорию России военных [КНДР...,Политика
4,\n\n\n\n\n\n\n Минобороны п...,Политика
...,...,...
1170,\n\n\n\n\n\n\n ЦБ подумает ...,Финансы
1171,В отличие от стратегий автоследования в ОПИФе ...,Финансы
1172,\n\n\n\n\n\n\n Индекс Мосби...,Финансы
1173,Идею привлечения авторов инвестиционных блогов...,Финансы


In [96]:
print(rbk_news.columns)
print(lenta_news.columns)
rbk_news.columns == lenta_news.columns

Index(['Text', 'topic'], dtype='object')
Index(['Text', 'topic'], dtype='object')


array([ True,  True])

In [97]:
all_news_data = pd.concat([lenta_news,rbk_news],axis = 0,ignore_index=True)

In [100]:
all_news_data

,Text,topic
0,МВД создало бюро по борьбе с отмыванием денег ...,Экономика
1,Правительство не будет занимать деньги у ЦБ Пр...,Экономика
2,"ВВС США заказали 60 ""Боингов"" для вторжения в ...",Экономика
3,Правительственная комиссия не дала ExxonMobil ...,Экономика
4,Неимущим для получения пособия придется расста...,Экономика
...,...,...
25119,\n\n\n\n\n\n\n ЦБ подумает ...,Финансы
25120,В отличие от стратегий автоследования в ОПИФе ...,Финансы
25121,\n\n\n\n\n\n\n Индекс Мосби...,Финансы
25122,Идею привлечения авторов инвестиционных блогов...,Финансы


In [103]:
labels = all_news_data['topic'].unique()
print(len(labels))
labels

7


array(['Экономика', 'Интернет и СМИ', 'Спорт', 'Культура', 'Политика',
       'Бизнес', 'Финансы'], dtype=object)

In [99]:
all_news_data.to_csv('all_news_data.csv')

In [117]:
for one_label in labels:
    the_one = all_news_data.loc[all_news_data['topic'] == one_label,'Text'].iloc[rd.randint(0,300)]
    print(one_label)
    print(the_one)
    print('\n\n')

Экономика
Касьянов заинтересовал Словакию новым газопроводом Словацкая сторона проявила "заметный интерес" к идее строительства газопровода по своей территории для транспортировки российского газа в Европу, об этом заявлено на встрече премьер-министра РФ Михаила Касьянова и министра экономики Словакии Любомира Гараха в среду в московском Доме правительства. Как сообщают из Департамента правительственной информации, Словакия уже заявила, что придает большое значение этому проекту, который свяжет магистральные газопроводы на территории Словакии и Польши. Сооружение новой нитки газопровода через Словакию позволит направить российский газ с Ямала в Европу в обход Украины, которая продолжает прибегать к несанкционированным заборам российского газа. Потери России от действий Украины оцениваются в несколько миллиардов кубических  метров в год. Беседа Касьянова и Гараха состоялась в рамках обсуждения итогов прошедшего накануне заседания российско-словацкой межправительственной комиссии по науч

In [9]:
all_news = pd.read_csv('all_news_data.csv',index_col = 0)

In [138]:
Train_news,Test_news = train_test_split(all_news,stratify=all_news['topic'],train_size=0.93)

In [139]:
print(len(Train_news))
print(len(Test_news))

23365
1759


In [ ]:
# Train_news.to_csv('Train_news.csv')
# Test_news.to_csv('Test_news.csv')

In [3]:
Train_news,Test_news = pd.read_csv('Train_news.csv',index_col=0),pd.read_csv('Test_news.csv',index_col=0)

In [ ]:
# Test_news = Test_news.reset_index().drop(['index'],axis = 1)
# Train_news = Train_news.reset_index().drop(['index'],axis = 1)

In [4]:
print(len(Train_news))
print(len(Test_news))

23364
1759


In [5]:
print(len(Train_news))
Train_news.iloc[:5,:]

23364


,Text,topic
0,"Украина готова вернуть ""взятый по ошибке"" кред...",Экономика
1,Рио-де-Жанейро стал кандидатом на роль столицы...,Спорт
2,Скончался драматург и сценарист Александр Воло...,Культура
3,Испанец пересек Атлантический океан на водном ...,Спорт
4,"Украина на Новый год останавливает все, что дв...",Интернет и СМИ


In [6]:
print(len(Test_news))
Test_news.iloc[:5,:]

1759


,Text,topic
0,"Падение фондового рынка возглавили ""ЮКОС"" и ""Г...",Экономика
1,Контрабанда нефти из Ирака достигла рекордного...,Экономика
2,"Андрей Шевченко продлил контракт с ""Миланом"" д...",Спорт
3,Американцы запрещают детям играть в компьютерн...,Интернет и СМИ
4,Napster идет маршем на Вашингтон Napster призы...,Интернет и СМИ


In [8]:
len(Test_news)

1759

In [13]:
len(Test_news.dropna())

1759

In [ ]:
# Train_news.dropna(inplace=True)
# Train_news.reset_index().drop('index',axis = 1,inplace = True)

In [ ]:
# Train_news.to_csv('Train_news.csv')

In [7]:
russian_stopwords = stopwords.words('russian')
print(len(russian_stopwords))
russian_stopwords[:10]

151


['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со']

In [8]:
def normalize_text(the_text):
    lowered = the_text.lower()
    found = re.findall(r'[А-я0-9A-z]+',lowered)
    filtered_found = [word for word in found if word != '' and word not in russian_stopwords]
    return filtered_found

In [20]:
example_text = Train_news.iloc[3,0]
example_text

'Испанец пересек Атлантический океан на\xa0водном мотоцикле Испанец Альваро де Маричалар (Alvaro de Marichalar) пересек Атлантический океан на водном мотоцикле, сообщает в воскресенье BBC News. Маричалар начал свое путешествие четыре месяца в Риме. Проплыв по реке Тибр до Средиземного моря, он делал остановки в Марокко, на Гибралтаре и на Канарских островах. Достигнув Атлантического океана, путешественник посетил Кубу и в Доминиканскую республику. Конечной точкой маршрута Маричалара стал Майами, США. Маричалар ехал 12 часов в сутки. На протяжении всего маршрута его сопровождало судно с командой из шести человек на борту, которое перевозило еду и топливо для водного мотоцикла. По словам испанца, он потратил на подготовку к путешествию несколько лет, и поправился на 18 килограмм, чтобы компенсировать потерю веса во время прохождения маршрута. Отвечая на вопрос журналистов, что было самым тяжелым во время пересечения Атлантического океана, Маричалар ответил: "Холод. По дороге из Рима в Не

In [20]:
def cleared_texts(text_col):
    for ith_row in range(len(text_col)):
        for token in normalize_text(text_col.iloc[ith_row]):
            yield token

In [22]:
tokens_dataset = cleared_texts(Train_news['Text'])

In [23]:
wordlevel_trainer = trainers.WordLevelTrainer(vocab_size = 500_000,show_progress = True,special_tokens = ['[UNK]','[PAD]'],min_frequency = 5)
wordlevel_model = models.WordLevel()
Train_tokenizer = Tokenizer(model = wordlevel_model)

In [24]:
Train_tokenizer.train_from_iterator(tokens_dataset,trainer=wordlevel_trainer)

In [ ]:
# Train_tokenizer.get_vocab_size()

57568

In [ ]:
# with open('Train_tokenizer.pkl','wb') as file:
#     pickle.dump(Train_tokenizer,file)

In [26]:
with open('Train_tokenizer.pkl','rb') as file:
    Train_tokenizer = pickle.load(file)

In [28]:
Train_tokenizer.get_vocab_size()

57568

In [65]:
test_tokens_dataset = cleared_texts(Test_news['Text'])

In [66]:
wordlevel_trainer = trainers.WordLevelTrainer(vocab_size = 500_000,show_progress = True,special_tokens = ['[UNK]','[PAD]'])
wordlevel_model = models.WordLevel()
Test_tokenizer = Tokenizer(model = wordlevel_model)

In [67]:
Test_tokenizer.train_from_iterator(test_tokens_dataset,trainer=wordlevel_trainer)

In [75]:
Test_tokenizer.get_vocab_size()

51219

In [ ]:
# with open('Test_tokenizer.pkl','wb') as file:
#     pickle.dump(Test_tokenizer,file)

In [9]:
#loading main functions
def normalize_text(the_text):
    lowered = the_text.lower()
    found = re.findall(r'[А-я0-9A-z]+',lowered)
    filtered_found = [word for word in found if word != '' and word not in russian_stopwords]
    return filtered_found


def cleared_texts(text_col):
    for ith_row in range(len(text_col)):
        for token in normalize_text(text_col.iloc[ith_row]):
            yield token


russian_stopwords = stopwords.words('russian')
print(len(russian_stopwords))
russian_stopwords[:10]

151


['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со']

In [10]:
#loading datasets
Train_news,Test_news = pd.read_csv('Train_news.csv',index_col=0),pd.read_csv('Test_news.csv',index_col=0)

In [11]:
#loading saved tokenizers
with open('Train_tokenizer.pkl','rb') as file:
    Train_tokenizer = pickle.load(file)

with open('Test_tokenizer.pkl','rb') as file:
    Test_tokenizer = pickle.load(file)

In [12]:
Train_news.iloc[:3,0]

0    Украина готова вернуть "взятый по ошибке" кред...
1    Рио-де-Жанейро стал кандидатом на роль столицы...
2    Скончался драматург и сценарист Александр Воло...
Name: Text, dtype: object

In [13]:
Test_news.iloc[:3,0]

0    Падение фондового рынка возглавили "ЮКОС" и "Г...
1    Контрабанда нефти из Ирака достигла рекордного...
2    Андрей Шевченко продлил контракт с "Миланом" д...
Name: Text, dtype: object

In [14]:
print(Test_tokenizer.get_vocab_size())
print(Train_tokenizer.get_vocab_size())

51219
57568


In [172]:
#train the embeddings (3 types)
#n_grams
embd_training_text_list = list(cleared_texts(Train_news['Text']))
embd_test_text_list = list(cleared_texts(Test_news['Text']))

In [16]:
print(embd_training_text_list[:5])
print(embd_test_text_list[:5])

['украина', 'готова', 'вернуть', 'взятый', 'ошибке']
['падение', 'фондового', 'рынка', 'возглавили', 'юкос']


In [17]:
def create_n_grams(list_of_tokens,window_size):
    '''creates N-grams with specified window'''
    for ind in range(window_size,len(list_of_tokens)-window_size):
        n_grams = []
        #adding the words in window to the list
        for window_gap in range(1,window_size + 1):
            n_grams.append(list_of_tokens[ind - window_gap])
        for window_gap in range(1,window_size + 1):
            n_grams.append(list_of_tokens[ind + window_gap])
        #yielding the result so that we can save some memory)
        yield n_grams,list_of_tokens[ind]

In [173]:
train_n_grams = create_n_grams(embd_training_text_list,window_size=2)
test_n_grams = create_n_grams(embd_test_text_list,window_size=2)

In [19]:
#example
print(next(train_n_grams))
print(next(train_n_grams))
print(next(train_n_grams))

(['готова', 'украина', 'взятый', 'ошибке'], 'вернуть')
(['вернуть', 'готова', 'ошибке', 'кредит'], 'взятый')
(['взятый', 'вернуть', 'кредит', 'мвф'], 'ошибке')


In [20]:
print(next(test_n_grams))
print(next(test_n_grams))
print(next(test_n_grams))

(['фондового', 'падение', 'возглавили', 'юкос'], 'рынка')
(['рынка', 'фондового', 'юкос', 'газпром'], 'возглавили')
(['возглавили', 'рынка', 'газпром', 'настоящее'], 'юкос')


In [21]:
embd_test_text_list[:10]

['падение',
 'фондового',
 'рынка',
 'возглавили',
 'юкос',
 'газпром',
 'настоящее',
 'время',
 '16',
 '00']

In [22]:
embd_training_text_list[:10]

['украина',
 'готова',
 'вернуть',
 'взятый',
 'ошибке',
 'кредит',
 'мвф',
 'украина',
 'готова',
 'вернуть']

In [23]:
class NGramDataset(Dataset):
    def __init__(self,tokenizer,the_data):
        self.main_data = the_data
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.main_data)
    
    def __getitem__(self,indx):
        X = torch.tensor([self.tokenizer.token_to_id(word) if self.tokenizer.token_to_id(word) != None else self.tokenizer.token_to_id('[UNK]') for word in self.main_data[indx][0]],dtype=torch.long)
        true_word = self.main_data[indx][1]
        y = self.tokenizer.token_to_id(true_word) if self.tokenizer.token_to_id(true_word) != None else self.tokenizer.token_to_id('[UNK]')
        return X,y

In [174]:
Train_Ngramsdataset = NGramDataset(Train_tokenizer,list(train_n_grams))

In [241]:
#saving this dataset
# with open('/home/luchian/all_data/uni_data/Train_Ngramsdataset','wb') as file:
#     pickle.dump(Train_Ngramsdataset,file)

In [266]:
#loading this dataset
with open('/home/luchian/all_data/uni_data/Train_Ngramsdataset','rb') as file:
    Train_Ngramsdataset = pickle.load(file)

In [267]:
print(len(Train_Ngramsdataset))
print(Train_Ngramsdataset[339823])

3153657
(tensor([8481,   69,  670,  370]), 140)


In [268]:
class NGramModel(nn.Module):
    def __init__(self,vocab_size,dim_embed,window_size = 2):
        super().__init__()
        self.embed = nn.Embedding(vocab_size,dim_embed)
        #from both sides
        self.linear1 = nn.Linear(2*window_size*dim_embed,128)
        self.linear2 = nn.Linear(128,vocab_size)
        self.rel = nn.ReLU()

    def forward(self,ind_list):
        y = self.embed(ind_list)
        y = y.reshape(y.shape[0],-1)
        y = self.rel(self.linear1(y))
        y = self.linear2(y)
        return y

In [269]:
#training
def train_ngram_model(the_model,epoch,loader,optimizer,loss_func,dev = 'cuda'):
    for one_ep in range(epoch):
        the_model.train()
        the_losses = []
        for X,y in tqdm(loader,desc = 'Going through the loader'):
            X,y = X.to(device = 'cuda'),y.to(device = 'cuda')
            optimizer.zero_grad()
            y_pred = the_model(X)
            the_loss = loss_func(y_pred,y)
            the_loss.backward()
            the_losses.append(the_loss.item())
            optimizer.step()
        if one_ep%1== 0:
            print(f'Current Loss: {the_losses[-1]}')

In [270]:
#setting the model
vocab_size = Train_tokenizer.get_vocab_size()
ngram_model = NGramModel(vocab_size,dim_embed = 100,window_size = 2)
ngram_model.to(device = 'cuda')

NGramModel(
  (embed): Embedding(57568, 100)
  (linear1): Linear(in_features=400, out_features=128, bias=True)
  (linear2): Linear(in_features=128, out_features=57568, bias=True)
  (rel): ReLU()
)

In [271]:
train_loader = DataLoader(dataset = Train_Ngramsdataset,shuffle = True,batch_size = 64)
lr = 0.03
epoch = 10
optimizer = torch.optim.SGD(lr = lr,params= ngram_model.parameters(),momentum=0.03)
the_loss = nn.CrossEntropyLoss()

In [272]:
#training ngrams models
train_ngram_model(ngram_model,epoch,train_loader,optimizer,the_loss,dev = 'cuda')

Going through the loader:   1%|          | 517/49276 [00:02<04:24, 184.56it/s]


KeyboardInterrupt: 

In [297]:
#CBOW model 
#we can use the same dataset here, but we just have to sum things up
class CBOWModel(nn.Module):
    def __init__(self,vocab_size,dim_embed,window_size = 2):
        super().__init__()
        self.embed = nn.Embedding(vocab_size,dim_embed)
        self.linear = nn.Linear(1,vocab_size)
        self.rel = nn.ReLU()

    def forward(self,ind_list):
        y = self.embed(ind_list).sum(dim = (1,2)).reshape(-1,1)
        y = self.rel(self.linear(y))
        return y

In [298]:
#setting the model
vocab_size = Train_tokenizer.get_vocab_size()
cbow_model = CBOWModel(vocab_size,dim_embed = 100,window_size = 2)
cbow_model.to(device = 'cuda')

CBOWModel(
  (embed): Embedding(57568, 100)
  (linear): Linear(in_features=1, out_features=57568, bias=True)
  (rel): ReLU()
)

In [299]:
train_cbow_loader = DataLoader(dataset = Train_Ngramsdataset,shuffle = True,batch_size = 64)
lr = 0.03
epoch = 10
optimizer = torch.optim.SGD(lr = lr,params= cbow_model.parameters(),momentum=0.03)
the_loss = nn.CrossEntropyLoss()

In [306]:
def train_cbow_model(the_model,epoch,loader,optimizer,loss_func,dev = 'cuda'):
    for one_ep in range(epoch):
        the_model.train()
        the_losses = []
        for X,y in tqdm(loader,desc = 'Going through the loader'):
            X,y = X.to(device = 'cuda'),y.to(device = 'cuda')
            optimizer.zero_grad()
            y_pred = the_model(X)
            the_loss = loss_func(y_pred,y)
            the_loss.backward()
            the_losses.append(the_loss.item())
            optimizer.step()
        if one_ep%1== 0:
            print(f'Current Loss: {the_losses[-1]}')

In [308]:
train_cbow_model(cbow_model,epoch,train_cbow_loader,optimizer,the_loss)

Going through the loader:   1%|          | 282/49276 [00:01<03:06, 262.54it/s]


KeyboardInterrupt: 

In [170]:
#encoding labels (they have to be the same in the dataset below for test and for train)
unique_train_labels = Train_news['topic'].unique()
unique_test_labels = Test_news['topic'].unique()
print(set(unique_test_labels) == set(unique_train_labels),end = '\n\n')
print(type(unique_test_labels))
print(unique_train_labels)
print(unique_test_labels)

True

<class 'numpy.ndarray'>
['Экономика' 'Спорт' 'Культура' 'Интернет и СМИ' 'Финансы' 'Бизнес'
 'Политика']
['Экономика' 'Спорт' 'Интернет и СМИ' 'Культура' 'Политика' 'Финансы'
 'Бизнес']


In [172]:
le = preprocessing.LabelEncoder().fit(unique_train_labels)
print(le.classes_)
print(le.transform(['Экономика']).item())

['Бизнес' 'Интернет и СМИ' 'Культура' 'Политика' 'Спорт' 'Финансы'
 'Экономика']
6


In [ ]:
class Padding(pad = 3):
    def __call__(tens):
        pass

In [191]:
some_text = normalize_text(Train_news.iloc[0,0])
len(some_text)

129

In [213]:
#creating dataset for word indices and padding
class NewsDataset(Dataset):
    def __init__(self,the_data,le,tokenizer,transforms = None):
        self.main_data = the_data
        self.tokenizer = tokenizer
        self.label_encoder = le
        self.transforms = transforms

    def __len__(self):
        return len(self.main_data)
    
    def __getitem__(self,indx):
        if self.transforms == None:
            X = [self.tokenizer.token_to_id(one_token)  if self.tokenizer.token_to_id(one_token) != None else self.tokenizer.token_to_id(['[UNK]']) for one_token in normalize_text(self.main_data.iloc[indx,0])]
            y = self.label_encoder.transform([self.main_data.iloc[indx,1]]).item()
            X,y = torch.tensor(X,dtype = torch.long),torch.tensor(y,dtype = torch.long)
            return X,y

In [214]:
TrainDataset = NewsDataset(Train_news,le,Train_tokenizer,transforms=None)

In [215]:
Train_news

,Text,topic
0,"Украина готова вернуть ""взятый по ошибке"" кред...",Экономика
1,Рио-де-Жанейро стал кандидатом на роль столицы...,Спорт
2,Скончался драматург и сценарист Александр Воло...,Культура
3,Испанец пересек Атлантический океан на водном ...,Спорт
4,"Украина на Новый год останавливает все, что дв...",Интернет и СМИ
...,...,...
23360,"Адвокаты ""ЮКОСа"" предложили план погашения дол...",Экономика
23361,Бывший вице-президент США собирается приобрест...,Интернет и СМИ
23362,Японский фондовый рынок обвалился до рекордног...,Экономика
23363,"""Электронная Россия"" обещает отечественным раз...",Интернет и СМИ


In [216]:
TrainDataset[0]

TypeError: argument 'token': 'list' object cannot be converted to 'PyString'

In [210]:
for one_token in TrainDataset[0]:
    print(one_token)
    if Train_tokenizer.token_to_id(one_token) == None:
        print(one_token)
        break
    print(Train_tokenizer.token_to_id(one_token),end = '\n\n')

украина
2160

готова
2286

вернуть
1705

взятый
30922

ошибке
11018

кредит
2287

мвф
941

украина
2160

готова
2286

вернуть
1705

200
607

миллионов
22

долларов
8

кредитных
2592

средств
339

полученных
3370

международного
904

валютного
3994

фонда
620

1997
771

году
9

основе
989

неправильных
54352

данных
707

состоянии
1210

валютных
6411

резервов
2223

украины
598

заявил
16

премьер
163

министр
151

украины
598

виктор
527

ющенко
16896

время
10

своей
62

поездки
5386

сша
14

передает
66

bbc
289

ющенко
16896

отметил
75

раздутые
раздутые
